In [18]:
import pandas as pd
import numpy as np
import json
import itertools
import scipy.stats as stats
import math

%matplotlib inline

In [19]:
before_normalization = pd.read_csv('../responses_data/responses.csv')

before_normalization['Q1_entropy'] = before_normalization['Q1_entropy'] * math.log2(20)
before_normalization['Q2_entropy'] = before_normalization['Q2_entropy'] * math.log2(20)

/usr/local/anaconda3/envs/asp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
# pd.concat([before_normalization['Q1_entropy'], before_normalization['Q2_entropy']]).describe()
# pd.concat([before_normalization['Q1_entropy'], before_normalization['Q2_entropy']]).plot.hist()

In [21]:
# normalize the data by ensuring that entropy q1 <= entropy q2.

# to_swap = before_normalization[before_normalization['Q1_entropy'] > before_normalization['Q2_entropy']]
# no_swap = before_normalization[before_normalization['Q1_entropy'] <= before_normalization['Q2_entropy']]

# swapped = to_swap.rename(index=str, columns={
#     'Q1_entropy': 'Q2_entropy',
#     'Q2_entropy': 'Q1_entropy',
#     'Q1_channel': 'Q2_channel',
#     'Q2_channel': 'Q1_channel',
#     'Q1_entropy_class': 'Q2_entropy_class',
#     'Q2_entropy_class': 'Q1_entropy_class'
# })

# normalized = pd.concat([no_swap, swapped])

normalized = before_normalization

In [22]:
# limit to one task

normalized = normalized[normalized['taskGroup'] == 'readValue']
len(normalized)

46080

In [23]:
# put entropy into quantiles (low, high)

normalized['Q1_entropy_group'] = pd.qcut(normalized['Q1_entropy'], 2, labels=["L", "H"])
normalized['Q2_entropy_group'] = pd.qcut(normalized['Q2_entropy'], 2, labels=["L", 'H'])

# entropy_cut = 0.3
# normalized.loc[normalized['Q1_entropy'] > entropy_cut, 'Q1_entropy_group'] = 'H'
# normalized.loc[normalized['Q1_entropy'] <= entropy_cut, 'Q1_entropy_group'] = 'L'
# normalized.loc[normalized['Q2_entropy'] > entropy_cut, 'Q2_entropy_group'] = 'H'
# normalized.loc[normalized['Q2_entropy'] <= entropy_cut, 'Q2_entropy_group'] = 'L'

/usr/local/anaconda3/envs/asp/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/anaconda3/envs/asp/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
gb = normalized.groupby([
    'taskGroup', 'cardinality', 'nPerCategory',
    'Q1_channel', 'Q2_channel', 'name_channel',
    'Q1_entropy_class', 'Q2_entropy_class',
#    'Q1_entropy_group', 'Q2_entropy_group'
])

In [25]:
row = normalized.iloc[0]
# row

In [26]:
def parse(group):
    cardinality = int(group['cardinality'].iloc[0])
    num_rows = cardinality * int(group['nPerCategory'].iloc[0])

    q1_entropy = group['Q1_entropy'].mean();
    q2_entropy = group['Q2_entropy'].mean();
    
    Q1_entropy_class = group['Q1_entropy_class'].iloc[0];
    Q2_entropy_class = group['Q2_entropy_class'].iloc[0];
    
    # we can identify the same dataset with the key
    data_key = f'{num_rows},{Q1_entropy_class},{Q2_entropy_class}'
    
    fields = [
        {'name': 'n', 'type': 'string', 'entropy': 1, 'cardinality': cardinality},
        {'name': 'q1', 'type': 'number', 'entropy': f'{q1_entropy:.3}', 'cardinality': num_rows, 'interesting': True},
        {'name': 'q2', 'type': 'number', 'entropy': f'{q2_entropy:.3}', 'cardinality': num_rows}
    ]

    enc = {}

    enc[group['Q1_channel'].iloc[0]] = {'field': 'q1', 'type': 'quantitative'}
    enc[group['Q2_channel'].iloc[0]] = {'field': 'q2', 'type': 'quantitative'}
    enc[group['name_channel'].iloc[0]] = {'field': 'n', 'type': 'nominal'}

    spec = {
        'mark': 'point',
        'encoding': enc
    }

    return {
        'data_key': data_key,
        'fields': fields,
        'num_rows': num_rows,
        'task': group['taskGroup'].iloc[0],
        'spec': spec,
        'score1': group['isCorrect'].mean(),
        'score2': group['completionTime'].mean(),
        'isCorrect': group['isCorrect'],
        'completionTime': group['completionTime'],
    }

In [27]:
specs = pd.DataFrame(columns=['data_key', 'fields', 'num_rows', 'task', 'spec',
                              'score1', 'score2', 'isCorrect', 'completionTime'])
for name, group in gb:
    parsed = parse(group)
    specs = specs.append(parsed, ignore_index=True)
    
print('Number of groups', len(specs))

Number of groups 288


In [28]:
specs = specs.sort_values(['score1', 'score2'])  # sort by correct ratio, then completion time
grouped_specs = specs.groupby(['data_key', 'num_rows', 'task'])

print(len(grouped_specs))

24


In [29]:
grouped_specs.describe()
# specs[:10]

score1                                         \
                             count      mean       std      min       25%   
data_key num_rows task                                                      
30,H,H   30       readValue   12.0  0.925521  0.066276  0.80625  0.879688   
30,H,L   30       readValue   12.0  0.930729  0.080325  0.75625  0.871875   
30,L,H   30       readValue   12.0  0.906250  0.041969  0.81250  0.898438   
30,L,L   30       readValue   12.0  0.921354  0.065358  0.73125  0.918750   
300,H,H  300      readValue   12.0  0.906771  0.078537  0.73125  0.885938   
300,H,L  300      readValue   12.0  0.904167  0.090819  0.76250  0.814062   
300,L,H  300      readValue   12.0  0.898958  0.067884  0.75000  0.881250   
300,L,L  300      readValue   12.0  0.847917  0.108231  0.58125  0.800000   
60,H,H   60       readValue   12.0  0.928646  0.029249  0.86250  0.910937   
60,H,L   60       readValue   12.0  0.884896  0.062809  0.73750  0.867188   
60,L,H   60       readValue   12.0  0.951563  0.036260  0.88125  0.940625   
60,L,L   60       readValue   12.0  0.888021  0.080104  0.70000  0.898438   
600,H,H  600      readValue   12.0  0.877083  0.096653  0.68750  0.829688   
600,H,L  600      readValue   12.0  0.935938  0.086772  0.78750  0.887500   
600,L,H  600      readValue   12.0  0.847917  0.101807  0.64375  0.793750   
600,L,L  600      readValue   12.0  0.851562  0.176498  0.47500  0.884375   
9,H,H    9        readValue   12.0  0.949479  0.035099  0.89375  0.918750   
9,H,L    9        readValue   12.0  0.878646  0.084168  0.70000  0.826563   
9,L,H    9        readValue   12.0  0.839062  0.045549  0.74375  0.804688   
9,L,L    9        readValue   12.0  0.906250  0.106700  0.61875  0.882812   
90,H,H   90       readValue   12.0  0.907292  0.129899  0.68125  0.790625   
90,H,L   90       readValue   12.0  0.947917  0.068431  0.80000  0.903125   
90,L,H   90       readValue   12.0  0.873437  0.105869  0.65625  0.859375   
90,L,L   90       readValue   12.0  0.861458  0.132524  0.64375  0.710938   

                                                         score2                \
                                  50%       75%      max  count          mean   
data_key num_rows task                                                          
30,H,H   30       readValue  0.956250  0.975000  0.98750   12.0   6502.733854   
30,H,L   30       readValue  0.975000  0.987500  0.99375   12.0   7389.208333   
30,L,H   30       readValue  0.909375  0.926562  0.96250   12.0   6732.746354   
30,L,L   30       readValue  0.928125  0.964063  0.97500   12.0   5790.259896   
300,H,H  300      readValue  0.937500  0.956250  1.00000   12.0   6933.938021   
300,H,L  300      readValue  0.934375  0.984375  0.99375   12.0   5822.175521   
300,L,H  300      readValue  0.928125  0.945312  0.96250   12.0   5651.873438   
300,L,L  300      readValue  0.900000  0.920312  0.93125   12.0   6068.411979   
60,H,H   60       readValue  0.934375  0.951562  0.96250   12.0   5897.493229   
60,H,L   60       readValue  0.906250  0.926563  0.95000   12.0   7321.098958   
60,L,H   60       readValue  0.956250  0.973437  0.99375   12.0   6976.336458   
60,L,L   60       readValue  0.925000  0.932813  0.94375   12.0   6289.536979   
600,H,H  600      readValue  0.909375  0.956250  0.96250   12.0   7419.972396   
600,H,L  600      readValue  0.984375  0.995313  1.00000   12.0   7876.072396   
600,L,H  600      readValue  0.884375  0.920312  0.96875   12.0   7525.784896   
600,L,L  600      readValue  0.937500  0.943750  0.96250   12.0   6762.954688   
9,H,H    9        readValue  0.953125  0.976562  1.00000   12.0  13657.314062   
9,H,L    9        readValue  0.915625  0.939063  0.95625   12.0   6424.356771   
9,L,H    9        readValue  0.856250  0.876563  0.88750   12.0   5582.235938   
9,L,L    9        readValue  0.950000  0.965625  0.98750   12.0   6181.983333   
90,H,H   90       readValue  0.987500  0.995313  1.00000   12.0   6166.327083   
90,H

In [30]:
training = []

for name, group in grouped_specs:
    # for neg, pos in zip(range(len(group) - 1), range(1, len(group))):
    for neg, pos in itertools.combinations(range(len(group)), 2):
        negative = group.iloc[neg]
        positive = group.iloc[pos]
        
        # run Welch's t-test
        t, p = stats.ttest_ind(negative.isCorrect, positive.isCorrect, equal_var=False)
#         if p > .05:
#             t, p = stats.ttest_ind(first.completionTime, second.completionTime, equal_var=False)
        if p <= .01:
            training.append({
                'fields': group.iloc[0]['fields'],
                'num_rows': group.iloc[0]['num_rows'],
                'task': group.iloc[0]['task'],
                'negative': negative.spec,
                'positive': positive.spec,
                'p-value': p
            })
    
#     g = list(group.iterrows())
#     previous = g[0][1]
#     for idx, row in g[1:]:
#         spec = row.spec
#         training.append({
#             'fields': row['fields'],
#             'num_rows': row['num_rows'],
#             'task': row['task'],
#             'worse': previous.spec,
#             'better': spec
#         })
#         previous = row

In [31]:
len(training)

591

In [32]:
training[0]

{'fields': [{'cardinality': 10, 'entropy': 1, 'name': 'n', 'type': 'string'},
  {'cardinality': 30,
   'entropy': '3.86',
   'interesting': True,
   'name': 'q1',
   'type': 'number'},
  {'cardinality': 30, 'entropy': '3.88', 'name': 'q2', 'type': 'number'}],
 'negative': {'encoding': {'size': {'field': 'q1', 'type': 'quantitative'},
   'x': {'field': 'q2', 'type': 'quantitative'},
   'y': {'field': 'n', 'type': 'nominal'}},
  'mark': 'point'},
 'num_rows': 30,
 'p-value': 0.0008957852184299649,
 'positive': {'encoding': {'size': {'field': 'q2', 'type': 'quantitative'},
   'x': {'field': 'n', 'type': 'nominal'},
   'y': {'field': 'q1', 'type': 'quantitative'}},
  'mark': 'point'},
 'task': 'readValue'}

In [33]:
with open('../data/training/q_q_n.json', 'w') as f:
    json.dump(training, f, indent=2)